In [ ]:
# !unzip -qq '/content/LG_aimers_B2BEST-main.zip' -d '/content/'

In [ ]:
!pip install --upgrade catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 2.8 MB/s eta 0:00:00


In [ ]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 4.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import re
import optuna
import matplotlib.pyplot as plt
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from collections import Counter
import catboost as cb
from catboost import CatBoostClassifier, Pool, cv
from sklearn.model_selection import train_test_split

In [ ]:
df_train = pd.read_csv("/content/train_pre.csv") # 학습용 데이터
df_test = pd.read_csv("/content/submission_pre.csv") # 테스트 데이터(제출파일의 데이터)

In [ ]:
def label_encoding(series: pd.Series) -> pd.Series:
    """범주형 데이터를 시리즈 형태로 받아 숫자형 데이터로 변환합니다."""

    my_dict = {}

    # 모든 요소를 문자열로 변환
    series = series.astype(str)

    for idx, value in enumerate(sorted(series.unique())):
        my_dict[value] = idx
    series = series.map(my_dict)

    return series

In [ ]:
# 레이블 인코딩할 칼럼들
label_columns = [
    "customer_country",
    "business_unit",
    "customer_type",
    "enterprise",
    "customer_job",
    "inquiry_type",
    "product_category",
    "product_subcategory",
    "product_modelname",
    "customer_position",
    "response_corporate",
    "expected_timeline",
    "business_area",
    "business_subarea",
    "category"
]

df_all = pd.concat([df_train[label_columns], df_test[label_columns]])

for col in label_columns:
    df_all[col] = label_encoding(df_all[col])

In [ ]:
for col in label_columns:
    df_train[col] = df_all.iloc[: len(df_train)][col]
    df_test[col] = df_all.iloc[len(df_train) :][col]

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(
    df_train.drop("is_converted", axis=1),
    df_train["is_converted"],
    test_size=0.2,
    shuffle=True,
    random_state=400,
)

In [ ]:
x_train

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,customer_job,lead_desc_length,...,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,category,product_count,timeline_count,idit_all
43573,0.50,61,0,NaN,5293,0,0,0.0,3,3,...,0,NaN,NaN,6,86,45,0,1,0,0
25857,0.75,74,2,0.049180,11045,4,0,0.0,15,70,...,1,0.001183,0.049840,10,32,153,1,3,0,0
42264,0.25,15,0,NaN,47466,0,0,16.0,19,145,...,0,NaN,NaN,6,86,40,3,1,0,0
7665,1.00,30,2,NaN,12526,0,0,0.0,19,5,...,0,NaN,NaN,6,86,175,1,1,0,0
23088,1.00,30,2,0.031196,40086,0,0,0.0,19,3,...,0,0.000572,0.048630,1,86,175,1,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,1.00,5,0,0.066667,39568,9,0,0.0,8,156,...,0,0.003079,0.026846,0,17,55,0,1,0,0
46009,0.50,10,0,NaN,35689,0,0,0.0,19,14,...,0,NaN,NaN,6,86,97,5,1,0,0
2366,0.75,30,2,NaN,25096,0,0,0.0,19,260,...,0,NaN,NaN,6,86,223,1,1,0,0
3535,0.50,30,2,NaN,5386,0,0,0.0,3,553,...,0,NaN,NaN,6,86,254,1,1,0,0


In [ ]:
def objective_catboost(trial):
    # 최적화할 파라미터 범위
    param = {
        'loss_function': 'MultiClass',  # 다중 분류 문제
        'iterations': 100,  # 트리의 개수, XGBoost의 'n_estimators'와 유사
        'depth': trial.suggest_int('depth', 1, 7),  # 트리의 깊이
        'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-3, 10.0),  # L2 정규화 계수
        'border_count': trial.suggest_int('border_count', 1, 255),  # 결정 경계의 개수
        'random_strength': trial.suggest_loguniform('random_strength', 1e-9, 10),  # 무작위성의 정도
        'random_state': 42
    }

    # CatBoost 모델 훈련
    model_cb = cb.CatBoostClassifier(**param, verbose=0)  # verbose=0으로 로깅 레벨을 조정
    model_cb.fit(x_train, y_train, eval_set=(x_val, y_val), use_best_model=True)

    # 검증 데이터셋에 대한 예측 및 정확도 계산
    preds = model_cb.predict(x_val)
    preds = np.vectorize(lambda x: x.lower())(model_cb.predict(x_val))
    accuracy = accuracy_score(y_val, preds)
    return accuracy


# Optuna 최적화
study = optuna.create_study(direction='maximize')
study.optimize(objective_catboost, n_trials=1000)

# 최적의 파라미터와 그때의 정확도 출력
print(f"Best trial: {study.best_trial.params}")
print(f"Best accuracy: {study.best_trial.value}")

best_params = study.best_trial.params
model_cat = cb.CatBoostClassifier(**best_params)

model_cat.fit(x_train, y_train)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
<ipython-input-22-8e322b715bce>:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-3, 10.0),  # L2 정규화 계수
<ipython-input-22-8e322b715bce>:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'random_strength': trial.suggest_loguniform('random_strength', 1e-9, 10),  # 무작위성의 정도
[I 2024-02-14 18:29:54,259] Trial 0 finished with value: 0.0 and parameters: {'depth': 7, 'l2_leaf_reg': 0.0030300429234811624, 'border_count': 232, 'random_strength': 7.917985655437808e-08}. Best is trial 0 with value: 0.0.
<ipython-input-22-8e322b715bce>:7: FutureWarning: suggest_loguniform has been deprecate

Best trial: {'depth': 7, 'l2_leaf_reg': 0.0030300429234811624, 'border_count': 232, 'random_strength': 7.917985655437808e-08}
Best accuracy: 0.0
0:	learn: 0.6317465	total: 18.9ms	remaining: 18.9s
1:	learn: 0.5766587	total: 38.1ms	remaining: 19s
2:	learn: 0.5298198	total: 56.9ms	remaining: 18.9s
3:	learn: 0.4835304	total: 91.5ms	remaining: 22.8s
4:	learn: 0.4481828	total: 110ms	remaining: 21.9s
5:	learn: 0.4208323	total: 128ms	remaining: 21.3s
6:	learn: 0.3956316	total: 147ms	remaining: 20.8s
7:	learn: 0.3696987	total: 164ms	remaining: 20.4s
8:	learn: 0.3502588	total: 182ms	remaining: 20.1s
9:	learn: 0.3290625	total: 200ms	remaining: 19.8s
10:	learn: 0.3134795	total: 218ms	remaining: 19.6s
11:	learn: 0.2996383	total: 235ms	remaining: 19.4s
12:	learn: 0.2853871	total: 254ms	remaining: 19.3s
13:	learn: 0.2742504	total: 272ms	remaining: 19.2s
14:	learn: 0.2590315	total: 291ms	remaining: 19.1s
15:	learn: 0.2498589	total: 323ms	remaining: 19.8s
16:	learn: 0.2417740	total: 341ms	remaining: 19

In [ ]:
cat_cols = x_train.columns[x_train.dtypes == 'object'].to_list() # 카테고리 칼럼
train_pool = Pool(x_train,y_train, cat_features=cat_cols)
eval_pool = Pool(x_val , y_val , cat_features=cat_cols)

# idit_all도 범주형 변수

param = {
    'depth': 7,
    'l2_leaf_reg': 0.0030300429234811624,
    'border_count': 232,
    'random_strength': 7.917985655437808e-08
}

model = CatBoostClassifier(**param)
model.fit(train_pool, eval_set=eval_pool)

0:	learn: 0.6317465	test: 0.6319887	best: 0.6319887 (0)	total: 65.1ms	remaining: 1m 4s
1:	learn: 0.5766587	test: 0.5770277	best: 0.5770277 (1)	total: 138ms	remaining: 1m 8s
2:	learn: 0.5298198	test: 0.5301540	best: 0.5301540 (2)	total: 196ms	remaining: 1m 5s
3:	learn: 0.4835304	test: 0.4837438	best: 0.4837438 (3)	total: 253ms	remaining: 1m 3s
4:	learn: 0.4481828	test: 0.4484949	best: 0.4484949 (4)	total: 305ms	remaining: 1m
5:	learn: 0.4208323	test: 0.4212446	best: 0.4212446 (5)	total: 373ms	remaining: 1m 1s
6:	learn: 0.3956316	test: 0.3961472	best: 0.3961472 (6)	total: 436ms	remaining: 1m 1s
7:	learn: 0.3696987	test: 0.3701384	best: 0.3701384 (7)	total: 525ms	remaining: 1m 5s
8:	learn: 0.3502588	test: 0.3510120	best: 0.3510120 (8)	total: 603ms	remaining: 1m 6s
9:	learn: 0.3290625	test: 0.3298243	best: 0.3298243 (9)	total: 639ms	remaining: 1m 3s
10:	learn: 0.3134795	test: 0.3144226	best: 0.3144226 (10)	total: 667ms	remaining: 59.9s
11:	learn: 0.2996383	test: 0.3006610	best: 0.3006610 (

In [ ]:
def get_clf_eval(y_test, y_pred=None):
    confusion = confusion_matrix(y_test, y_pred, labels=[True, False])
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, labels=[True, False])
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred, labels=[True, False])

    print("오차행렬:\n", confusion)
    print("\n정확도: {:.4f}".format(accuracy))
    print("정밀도: {:.4f}".format(precision))
    print("재현율: {:.4f}".format(recall))
    print("F1: {:.4f}".format(F1))

In [ ]:
# 테스트 데이터로 예측
pred = model.predict(x_val.fillna(0))
pred = [val.lower() == 'true' for val in pred]

In [ ]:
pred = model_cat.predict(x_val.fillna(0))
pred = [val.lower() == 'true' for val in pred]

In [ ]:
get_clf_eval(y_val, pred)

오차행렬:
 [[  685   262]
 [   63 10850]]

정확도: 0.9726
정밀도: 0.9158
재현율: 0.7233
F1: 0.8083


### 테스트 데이터 예측

In [ ]:
# 예측에 필요한 데이터 분리
x_test = df_test.drop(["is_converted", "id"], axis=1)

In [ ]:
test_pred = model.predict(x_test.fillna(0))
test_pred = [val.lower() == 'true' for val in test_pred]

[False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, True, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, True, False, True, False, False, False, True, False, False, False, False, False, True, False, True, False, False, False, True, False, False, False, True, False, False, False, False, False, False, False, True, False, False, False, False, True, False, False, F

In [ ]:
sum(test_pred) # True로 예측된 개수

540

### 제출 파일 작성

In [ ]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("submission_pre.csv")
df_sub["is_converted"] = test_pred

# 제출 파일 저장
df_sub.to_csv("submission2.csv", index=False)

**우측 상단의 제출 버튼을 클릭해 결과를 확인하세요**